In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('adult_with_pii.csv')

In [2]:
# import matplotlib.pyplot as plt
# print(plt.style.available)

# Question 1

Implement the dp_occupation_histogram function below. It should return a differentially private histogram over the Occupation column in the adult dataset. Your function should have a total privacy cost of epsilon and should use parallel composition.
(use laplace mechanism)

In [3]:
def dp_occupation_histogram(epsilon):
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    # Calculate sensitivity
    sensitivity = 1  # Sensitivity of counting queries

    # Compute the non-private histogram
    non_private_histogram = adult['Occupation'].value_counts().to_dict()

    # Add Laplace noise to each count
    dp_histogram = {}
    for occupation, count in non_private_histogram.items():
        noisy_count = laplace_mech(count, sensitivity, epsilon)
        dp_histogram[occupation] = noisy_count

    return dp_histogram

dp_occupation_histogram(1.0)

{'Prof-specialty': 4140.122095634974,
 'Craft-repair': 4099.399509080263,
 'Exec-managerial': 4064.7498355752455,
 'Adm-clerical': 3771.9462480759767,
 'Sales': 3650.5159122066657,
 'Other-service': 3295.664737648097,
 'Machine-op-inspct': 2001.1383590242187,
 'Transport-moving': 1597.692791407002,
 'Handlers-cleaners': 1370.4671878252873,
 'Farming-fishing': 993.7704842362438,
 'Tech-support': 928.357711358345,
 'Protective-serv': 649.5887650736229,
 'Priv-house-serv': 147.02346388150212,
 'Armed-Forces': 8.63443057503427,
 'Baby': -0.41350212649013907}

\begin{equation}
\text{Let's denote the differentially private histogram of the 'Occupation' column in the adult dataset with a privacy parameter (epsilon) of 1.0 as } H_{\text{DP}}^{(1)}, \text{ and the true histogram as } H_{\text{True}}. \text{ The absolute difference between the elements of these histograms is denoted as } \Delta H^{(1)} = | H_{\text{DP}}^{(1)} - H_{\text{True}} |. \\
\text{Now, considering a set of } N = 200 \text{ repetitions, we can represent the expression as a sequence:} \\
\Delta H^{(1)}_1, \Delta H^{(1)}_2, \Delta H^{(1)}_3, \ldots, \Delta H^{(1)}_N \\
\text{Where each } \Delta H^{(1)}_i \text{ represents the absolute differences between the differentially private histogram and the true histogram for the } i^{th} \text{ repetition.} \\
\text{Mathematically, each } \Delta H^{(1)}_i \text{ can be calculated using the formula:} \\
\Delta H^{(1)}_i = | H_{\text{DP}}^{(1)} - H_{\text{True}} | \\
\text{Finally, the entire expression can be written as a list comprehension:} \\
\text{dp_results} = [ \Delta H^{(1)}_i \text{ for } i \text{ in range}(N) ] \\
\end{equation}


Based on above answer these questions:

1) Generate 200 differentially private histograms with epsilon value=1

2)Compute the DP results "dp_rseults"

3)Flatten the list of histograms into a single list of absolute differences

4)Generate a sample of Laplace noise with scale = 1/1.0 and generate a list of 2000 samples

5)What is the  Wasserstein distance between the differential private results list above and the Laplace noise, what  is the integer upper bound

6)can you  assert that the Wasserstein distance between the differential private results and the Laplace noise is greater than 0,write the code

7)If you partition the data by both occupation and age (i.e. a contingency table), would parallel composition still apply? Why or why not?



Your answer to each question

In [4]:
epsilon = 1.0
dp_histograms = [dp_occupation_histogram(epsilon) for _ in range(200)]

In [5]:
true_histogram = adult['Occupation'].value_counts().to_dict()
dp_results = [{occupation: abs(dp_histogram[occupation] - true_histogram.get(occupation, 0)) for occupation in set(dp_histogram) | set(true_histogram)} for dp_histogram in dp_histograms]

In [6]:
absolute_differences = [count for dp_result in dp_results for count in dp_result.values()]

In [7]:
laplace_noise = np.random.laplace(loc=0, scale=1/epsilon, size=2000)

In [8]:
wasserstein_distance = stats.wasserstein_distance(absolute_differences, laplace_noise)
upper_bound = int(np.ceil(wasserstein_distance))
print("Wasserstein distance:", wasserstein_distance)
print("Integer upper bound:", upper_bound)

Wasserstein distance: 0.9338226079387666
Integer upper bound: 1


In [9]:
assert wasserstein_distance > 0, "Wasserstein distance is not greater than 0"

### Q 1.7
If you partition the data by both occupation and age (i.e., a contingency table), parallel composition would still apply. This is because parallel composition applies to independent computations over disjoint subsets of the data. As long as the privacy guarantees hold for each subset individually, they can be combined using parallel composition. However, if there are dependencies between the subsets (e.g., correlations between occupation and age), parallel composition may not apply directly. In such cases, more sophisticated privacy analysis techniques would be needed.

# Question 2

In [10]:

#Consider the code below, which defines three average queries and runs them on adult_data, using the Laplace mechanism to provide differential privacy with for each query.

b_capgain = 10000
b_age = 3000

epsilon = 1

def query1():
    return np.sum(adult['Capital Gain'].clip(lower=0, upper=b_capgain))

def query2():
    return len(adult[adult['Education-Num'] < 10])

def query3():
    return np.sum(adult['Age'].clip(lower=0, upper=b_age))

def my_query():
    return [query1(), query2(), query3()]

my_query()

[17145231, 14755, 1360238]

In 2-5 sentences, answer the following:

1)What is the L_1
 global sensitivity of my_query, and why?
2)What is the L_2
 global sensitivity of my_query, and why?

3)Can you release my_query with DP using paraellel compossition with epsilon =1( use L_1 sensitivity), write the code



1. The L1 global sensitivity of my_query is the maximum absolute difference between the outputs of my_query on any pair of neighboring datasets, where neighboring datasets differ by at most one individual's data entry. For my_query, the maximum absolute difference in outputs occurs when one individual's data entry changes in either the 'Capital Gain' or 'Age' column. Therefore, the L1 global sensitivity is the maximum absolute change in the sum of 'Capital Gain' or 'Age' due to a single individual's data entry, which can be bounded by b_capgain or b_age, respectively.

2. The L2 global sensitivity of my_query is the square root of the sum of squares of the individual sensitivities of each query function. Since each query function operates on a single column of the dataset, the L2 sensitivity of each query is the same as the L1 sensitivity (i.e., b_capgain for query1 and query3, and 1 for query2). Therefore, the L2 global sensitivity of my_query is the square root of the sum of squares of b_capgain and 1.

3. Yes, my_query can be released with differential privacy using parallel composition with epsilon = 1 and L1 sensitivity. We can add Laplace noise with scale equal to the L1 sensitivity of my_query. Here's the code:

In [11]:
sensitivity = max(b_capgain, b_age)  # L1 sensitivity
epsilon = 1
noisy_result = laplace_mech(np.array(my_query()), sensitivity, epsilon)
print(noisy_result.tolist())

[17134709.47030449, 4233.470304490036, 1349716.47030449]


# Question 3

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
X = np.load('adult_processed_x.npy')
y = np.load('adult_processed_y.npy')

In [14]:
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

y_test.shape

(9044,)

In [15]:
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

In [16]:
def avg_grad(theta, X, y):
    grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    return np.mean(grads, axis=0)

In [17]:
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

In [18]:
def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

# Implement noisy_gradient_descent_RDP, a variant of noisy gradient descent that uses Rényi differential privacy. Your solution should have a total privacy cost of $(\alpha, \bar{\epsilon})$-RDP.

In [28]:
def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def noisy_gradient_descent_RDP(iterations, alpha, epsilon_bar):
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    # Initial guess for theta
    theta = np.zeros(X_train.shape[1])  # Assuming theta is of the same dimension as the features
    
    for t in range(iterations):
        # Compute noisy average gradient using Gaussian mechanism with RDP
        noisy_avg_grad = gaussian_mech_RDP_vec(avg_grad(theta, X_train, y_train), 1, alpha, epsilon_bar / iterations)
        
        # Update theta
        theta = theta - noisy_avg_grad
        
    return theta

theta = noisy_gradient_descent_RDP(10, 20, 0.1)
print('Final accuracy:', accuracy(theta))  # use accuracy same as used in the book

Final accuracy: 0.6636444051304733


# Implement noisy_gradient_descent_zCDP, a variant of noisy gradient descent that uses zero-concentrated differential privacy. Your solution should have a total privacy cost of -zCDP.

In [22]:
def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def noisy_gradient_descent_zCDP(iterations, rho):
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    # Initial guess for theta
    theta = np.zeros(X.shape[1])  # Assuming theta is of the same dimension as the features
    
    for t in range(iterations):
        # Compute noisy average gradient using Gaussian mechanism with zCDP
        noisy_avg_grad = gaussian_mech_zCDP_vec(avg_grad(theta, X_train, y_train), 1, rho)
        
        # Update theta
        theta = theta - noisy_avg_grad
        
    return theta


theta = noisy_gradient_descent_zCDP(10, 0.1)
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.7393852277753207


Which of the following functions is likely to produce the best accuracy for a given privacy cost, and why? Which is likely to produce the worst accuracy for a given privacy cost, and why?( use the same dataset as used in notebook for noisy gradient descent for all cases)


- noisy_gradient_descent
- noisy_gradient_descent_RDP
- noisy_gradient_descent_zCDP

YOUR ANSWER HERE

To assess which function is likely to produce the best accuracy for a given privacy cost, and which is likely to produce the worst accuracy, let's consider the privacy properties of each mechanism:

**1. noisy_gradient_descent:**

- This mechanism adds Gaussian noise directly to the gradients without explicitly considering differential privacy. Therefore, it provides no formal privacy guarantee.
As a result, it may offer the best accuracy in terms of utility since it doesn't introduce any additional noise beyond what is necessary for optimization. However, it lacks any privacy guarantee.

**2. noisy_gradient_descent_RDP:**

- This mechanism uses the Gaussian mechanism with Rényi Differential Privacy (RDP), providing a formal privacy guarantee.
By using RDP, it ensures that the privacy cost is quantifiable and controllable, offering a balance between privacy and utility. However, the additional noise introduced for privacy may degrade accuracy compared to noisy_gradient_descent.

**3. noisy_gradient_descent_zCDP:**

- This mechanism uses the Gaussian mechanism with Zero-Concentrated Differential Privacy (zCDP), which is a relaxed notion of differential privacy.
zCDP provides a less stringent privacy guarantee compared to RDP, which may result in less noise being added for a given privacy cost.
However, zCDP does not provide as strong privacy guarantees as RDP, which may lead to a trade-off between privacy and utility. In some cases, this mechanism may offer better accuracy than RDP for a given privacy cost, but it may also be less robust in terms of privacy protection.


Considering these points, the function likely to produce the best accuracy for a given privacy cost is noisy_gradient_descent, as it does not introduce additional noise beyond what is necessary for optimization. However, it offers no formal privacy guarantee.

On the other hand, the function likely to produce the worst accuracy for a given privacy cost is noisy_gradient_descent_RDP. While it provides a formal privacy guarantee through RDP, the additional noise introduced for privacy may degrade accuracy compared to noisy_gradient_descent.

 # References

 https://programming-dp.com/book.pdf

 https://colab.research.google.com/drive/1wODT0dOa4Jd7b5X4L9aN8Es1MfzvuWAR#scrollTo=TqaQvOMQRx4U
 ( class notebook part2 )
